In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation,rc
import seaborn as sns
from IPython.display import HTML

# Verlet

In [2]:
def a(r,Rs,G,Ms):
    acceleration=0
    for i,R in enumerate(Rs):
        d=(r-R)
        acceleration+= -G*Ms*d/np.linalg.norm(d)**3
    return acceleration

In [3]:
r1=np.array([-1,0])
r3=np.array([0,0])

r0s=[r1,-r1,r3]

def positions(r0s,v0s,bt,Ms=1,G=1,ht=1e-3,h=1e-3):
    rs=[[r] for r in r0s]
    vs=[[v] for v in v0s]
    for i,r in enumerate(r0s):
        rsem=r0s[:i]+r0s[i+1:]
        rnew=r+h*v0s[i]+h**2*a(r,rsem,G,Ms)/2
        rs[i].append(rnew)

    ts=np.arange(0,bt/3,ht)
    for _ in ts[1:]:
        ra=[r[-1] for r in rs]
        for i, r in enumerate(ra):
            rsem=ra[:i]+ra[i+1:]
            rnew=2*r-rs[i][-2]+h**2*a(r,rsem,G,Ms)
            rs[i].append(rnew)
    rs=[np.array(r) for r in rs]
    return rs

# Animando Verlet

In [4]:
def animar_v(r1v,r2v,r3v,bt,v0s,ht=1e-3,h=1e-3,ylim=(-2,2),fsize=None):
  fig,ax=plt.subplots(figsize=fsize)

  ax.set_xlim((-2,2))
  ax.set_ylim(ylim)
  ax.grid()
  plt.close()

  x1=np.array([r[0] for r in r1v])
  y1=np.array([r[1] for r in r1v])
  x2=np.array([r[0] for r in r2v])
  y2=np.array([r[1] for r in r2v])
  x3=np.array([r[0] for r in r3v])
  y3=np.array([r[1] for r in r3v])
  line1,=ax.plot(x1,y1,lw=2)
  line2,=ax.plot(x2,y2,lw=2)
  line3,=ax.plot(x3,y3,lw=2)
  body1,=ax.plot(x1[0],y1[0],c='b',marker='o')
  body2,=ax.plot(x2[0],y2[0],c='y',marker='o')
  body3,=ax.plot(x3[0],y3[0],c='g',marker='o')

  def init():
      body1.set_data([],[])
      body2.set_data([],[])
      body3.set_data([],[])
      return (line1,line2,line3,body1,body2,body3)

  def animate(i):
      k=(bt/h)/(100*3)*i
      body1.set_data([x1[int(k)]],[y1[int(k)]])
      body2.set_data([x2[int(k)]],[y2[int(k)]])
      body3.set_data([x3[int(k)]],[y3[int(k)]])
      return(body1,body2,body3,)

  anim=animation.FuncAnimation(fig,animate,init_func=init,frames=100,interval=70,blit=True)
  return HTML(anim.to_html5_video())


No. 2

In [5]:
v1=np.array([0.322184765624991, 0.647989160156249])
v0s=[v1,v1,-2*v1]
bt=51.35
r1v,r2v,r3v=positions(r0s,v0s,bt,ht=1e-3,h=1e-3,Ms=1)

In [6]:
animar_v(r1v,r2v,r3v,bt,v0s)

No. 3

In [7]:
v1=np.array([0.257841699218752, 0.687880761718747])
v0s=[v1,v1,-2*v1]
bt=55.64
r1v,r2v,r3v=positions(r0s,v0s,bt,ht=1e-3,h=1e-3,Ms=1)

In [8]:
animar_v(r1v,r2v,r3v,bt,v0s)

No. 4

In [9]:
v1=np.array([0.568991007042164, 0.449428951346711])
v0s=[v1,v1,-2*v1]
bt=51.96
r1v,r2v,r3v=positions(r0s,v0s,bt,ht=1e-3,h=1e-3,Ms=1)

In [10]:
animar_v(r1v,r2v,r3v,bt,v0s)

No. 22

In [11]:
v1=np.array([0.698073236083981, 0.328500769042967])
v0s=[v1,v1,-2*v1]
bt=100.8
r1v,r2v,r3v=positions(r0s,v0s,bt,ht=1e-3,h=1e-3,Ms=1)

In [12]:
animar_v(r1v,r2v,r3v,bt,v0s,ylim=(-4,4),fsize=(4,8))

# RK4

In [3]:
def f(u_main,u_sem,G=1,M=1):
    """
    umain é vetor
    u_sem é a lista com vetores posiçao e velocidade sem umain
    f retorna a dupla de vetores, que são a derivada da dupla de vetores iniciais u_main
    """
    a=0
    for u in u_sem:
        d=u_main[0]-u[0]
        a+=-G*M*(d)/np.linalg.norm(d)**3
    derivada=np.array([u_main[1],a])
    return derivada

In [4]:
r1=np.array([-1,0])
r3=np.array([0,0])

r0s=[r1,-r1,r3]

def rk_positions(u1,u2,u3,bt,h=1e-3,ht=1e-3,G=1,M=1):
    ts=np.arange(0,bt/3,ht)
    us=[[u1],[u2],[u3]]
    u_pos=[[u[0][0]] for u in us]
    for _ in ts:
        ua=[u[-1] for u in us]
        for i,u in enumerate(ua):
            u_sem=ua[:i]+ua[i+1:]
            k1=f(u,u_sem)
            k2=f(u+h*k1/2,u_sem)
            k3=f(u+h*k2/2,u_sem)
            k4=f(u+h*k3,u_sem)
            unew=u+h*(k1+2*k2+2*k3+k4)/6
            us[i].append(unew)
            u_pos[i].append(unew[0])
    return np.array(u_pos)



# Animando RK4

In [5]:
def rk_animate(r1k,r2k,r3k,bt,ht=1e-4,h=1e-4,ylim=(-2,2),fsize=None,quadros=100):
  fig,ax=plt.subplots(figsize=fsize)

  ax.set_xlim((-2,2))
  ax.set_ylim(ylim)
  ax.grid()
  plt.close()

  x1=np.array([r[0] for r in r1k])
  y1=np.array([r[1] for r in r1k])
  x2=np.array([r[0] for r in r2k])
  y2=np.array([r[1] for r in r2k])
  x3=np.array([r[0] for r in r3k])
  y3=np.array([r[1] for r in r3k])
  line1,=ax.plot(x1,y1,lw=2)
  line2,=ax.plot(x2,y2,lw=2)
  line3,=ax.plot(x3,y3,lw=2)
  body1,=ax.plot(x1[0],y1[0],c='b',marker='o')
  body2,=ax.plot(x2[0],y2[0],c='y',marker='o')
  body3,=ax.plot(x3[0],y3[0],c='g',marker='o')

  def init():
      body1.set_data([],[])
      body2.set_data([],[])
      body3.set_data([],[])
      return (line1,line2,line3,body1,body2,body3)

  def animate(i):
      k=(bt/ht)/(quadros*3)*i
      body1.set_data([x1[int(k)]],[y1[int(k)]])
      body2.set_data([x2[int(k)]],[y2[int(k)]])
      body3.set_data([x3[int(k)]],[y3[int(k)]])
      return(body1,body2,body3,)

  anim=animation.FuncAnimation(fig,animate,init_func=init,frames=quadros,interval=70,blit=True)
  return HTML(anim.to_html5_video())


No. 2

In [21]:
v1=np.array([0.322184765624991, 0.647989160156249])
bt=51.59
u1=[r1,v1]
u2=[-r1,v1]
u3=[r3,-2*v1]

r1k,r2k,r3k=rk_positions(u1,u2,u3,bt,h=5e-5,ht=5e-5)

In [22]:
rk_animate(r1k,r2k,r3k,bt,h=5e-5,ht=5e-5)

No. 3

In [19]:
v1=np.array([0.257841699218752, 0.687880761718747])
bt=56.15
u1=[r1,v1]
u2=[-r1,v1]
u3=[r3,-2*v1]

r1k,r2k,r3k=rk_positions(u1,u2,u3,bt,h=2e-5,ht=2e-5)


In [20]:
rk_animate(r1k,r2k,r3k,bt,h=2e-5,ht=2e-5)

No. 4

In [7]:
v1=np.array([0.568991007042164, 0.449428951346711])
bt=51.98
u1=[r1,v1]
u2=[-r1,v1]
u3=[r3,-2*v1]

r1k,r2k,r3k=rk_positions(u1,u2,u3,bt,h=1e-4,ht=1e-4)


In [8]:
rk_animate(r1k,r2k,r3k,bt)

No. 22

In [5]:
v1=np.array([0.698073236083981, 0.328500769042967])
bt=101.1
u1=[r1,v1]
u2=[-r1,v1]
u3=[r3,-2*v1]

r1k,r2k,r3k=rk_positions(u1,u2,u3,bt,h=2e-5,ht=2e-5)


In [6]:
rk_animate(r1k,r2k,r3k,bt,fsize=(4,8),ylim=(-4,4),h=2e-5,ht=2e-5,quadros=150)